In [3]:
import torch
import sbi.utils as utils
from sbi.inference.base import infer

In [4]:
prior = utils.BoxUniform(
    low=torch.tensor([.05, .05, .05]),
    high=torch.tensor([.95, .95, .95])
)

In [5]:
prior

BoxUniform(Uniform(low: torch.Size([3]), high: torch.Size([3])), 1)

In [14]:
def simulator(rates):
    
    # Set species values
#     s_start = pyro.sample(
#         "start", 
#         pyro.distributions.Normal(10, 0.01).expand([3]).to_event(1))
    
    s_start = torch.tensor([10.,10.,10.])
    ## Loop over sequences
    batch = torch.tensor([[10.,10.,10.]] * 10)
    
    for t in torch.arange(1, 10):
        
        probs = s_start*rates
        probs = probs / probs.sum()
        x = torch.multinomial(probs, 1)

        update = torch.zeros(3)
        update[x] = 1.

    #with tones_plate:
        #sample = torch.normal(s_start + update, .01)
        batch[t] = s_start + update
        s_start = s_start + update
    
    return batch


In [15]:
simulator(torch.tensor([.5,.2,.9]))

tensor([[10., 10., 10.],
        [10., 10., 11.],
        [10., 10., 12.],
        [11., 10., 12.],
        [12., 10., 12.],
        [12., 11., 12.],
        [12., 11., 13.],
        [12., 12., 13.],
        [13., 12., 13.],
        [13., 12., 14.]])

In [ ]:
num_sim = 2000
method = 'SNPE' #SNPE or SNLE or SNRE
posterior = infer(
    simulator,
    prior,
    # See glossary for explanation of methods.
    #    SNRE newer than SNLE newer than SNPE.
    method=method,
    #num_workers=-1,
    num_simulations=num_sim)

ValueError: only one element tensors can be converted to Python scalars

In [7]:
num_dim = 3
prior = utils.BoxUniform(low=-2*torch.ones(num_dim), high=2*torch.ones(num_dim))

def simulator(parameter_set):
    return torch.normal(torch.tensor([1.0]), .01)
    #return 1.0 + parameter_set + torch.randn(parameter_set.shape) * 0.1

posterior = infer(simulator, prior, method='SNPE', num_simulations=1000)

Running 1000 simulations.:   0%|          | 0/1000 [00:00<?, ?it/s]

Neural network successfully converged after 95 epochs.
